In [1]:
!pip install optuna
!pip install --upgrade --no-cache-dir catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 42.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [4]:
!pip uninstall -y numpy tensorflow catboost
!pip install numpy==1.23.5 tensorflow==2.12.0 catboost --no-cache-dir

INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 242.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.0/586.0 MB 117.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 215.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 296.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 312.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 277.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 247.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 293.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 124.9 MB/s eta 0:00:

In [2]:
import numpy as np
print("NumPy:", np.__version__)

from tensorflow.keras.models import Model
from catboost import CatBoostClassifier

print("Everything is working!")

NumPy: 1.23.5
Everything is working!


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score, average_precision_score

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, UpSampling1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier

import optuna


# Load and preprocess dataset
df = pd.read_csv('creditcard.csv')
X = df.drop(['Time', 'Class'], axis=1)
y = df['Class']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply SMOTE
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_scaled, y)

# Pad features to length 32 (must be divisible by 4 for CAE symmetry)
pad_len = 32
X_pad = pad_sequences(X_res, maxlen=pad_len, padding='post')
X_cae = X_pad.reshape((X_pad.shape[0], X_pad.shape[1], 1))

# Build CAE model
def build_cae(input_shape):
    input_layer = Input(shape=input_shape)
    x = Conv1D(16, 3, activation='relu', padding='same')(input_layer)
    x = MaxPooling1D(2, padding='same')(x)
    x = Conv1D(8, 3, activation='relu', padding='same')(x)
    encoded = MaxPooling1D(2, padding='same')(x)
    x = Conv1D(8, 3, activation='relu', padding='same')(encoded)
    x = UpSampling1D(2)(x)
    x = Conv1D(16, 3, activation='relu', padding='same')(x)
    x = UpSampling1D(2)(x)
    decoded = Conv1D(1, 3, activation='sigmoid', padding='same')(x)
    cae = Model(input_layer, decoded)
    cae.compile(optimizer='adam', loss='mse')
    encoder = Model(input_layer, encoded)
    return cae, encoder

# Train CAE
cae, encoder = build_cae(X_cae.shape[1:])
cae.fit(X_cae, X_cae, epochs=10, batch_size=256, shuffle=True, verbose=1,
        callbacks=[EarlyStopping(patience=2, restore_best_weights=True)])

# Extract encoded features
X_encoded = encoder.predict(X_cae)
X_flattened = X_encoded.reshape((X_encoded.shape[0], -1))

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_flattened, y_res, test_size=0.2, random_state=42)

# Define optimization and models
def optimize_model(model_type):
    def objective(trial):
        if model_type == 'xgb':
            model = XGBClassifier(
                n_estimators=trial.suggest_int("n_estimators", 50, 200),
                max_depth=trial.suggest_int("max_depth", 3, 10),
                learning_rate=trial.suggest_float("learning_rate", 0.01, 0.3),
                use_label_encoder=False,
                eval_metric='logloss'
            )
        elif model_type == 'lgb':
            model = LGBMClassifier(
                n_estimators=trial.suggest_int("n_estimators", 50, 200),
                max_depth=trial.suggest_int("max_depth", 3, 10),
                learning_rate=trial.suggest_float("learning_rate", 0.01, 0.3)
            )
        else:
            model = CatBoostClassifier(
                iterations=trial.suggest_int("iterations", 50, 200),
                depth=trial.suggest_int("depth", 3, 10),
                learning_rate=trial.suggest_float("learning_rate", 0.01, 0.3),
                verbose=0
            )
        model.fit(X_train, y_train)
        preds = model.predict_proba(X_test)[:, 1]
        return average_precision_score(y_test, preds)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)
    return study.best_params

# Optimize models
params_xgb = optimize_model("xgb")
params_lgb = optimize_model("lgb")
params_cat = optimize_model("cat")

# Initialize models
xgb = XGBClassifier(**params_xgb, use_label_encoder=False, eval_metric='logloss')
lgb = LGBMClassifier(**params_lgb)
cat = CatBoostClassifier(**params_cat, verbose=0)

# Train ensemble
ensemble = VotingClassifier(estimators=[
    ('xgb', xgb),
    ('lgb', lgb),
    ('cat', cat)
], voting='soft')

ensemble.fit(X_train, y_train)

# Evaluate
y_pred_proba = ensemble.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
auprc = average_precision_score(y_test, y_pred_proba)

print(f"AUC: {auc:.4f}")
print(f"AUPRC: {auprc:.4f}")


Epoch 1/10
2219/2222 [============================>.] - ETA: 0s - loss: 9.0062

2222/2222 [==============================] - 41s 17ms/step - loss: 9.0070
Epoch 2/10
2221/2222 [============================>.] - ETA: 0s - loss: 8.8632

2222/2222 [==============================] - 38s 17ms/step - loss: 8.8633
Epoch 3/10
2221/2222 [============================>.] - ETA: 0s - loss: 8.8547

2222/2222 [==============================] - 39s 17ms/step - loss: 8.8547
Epoch 4/10
2221/2222 [============================>.] - ETA: 0s - loss: 8.8525

2222/2222 [==============================] - 39s 18ms/step - loss: 8.8528
Epoch 5/10
2221/2222 [============================>.] - ETA: 0s - loss: 8.8520

...

[LightGBM] [Info] Number of positive: 227339, number of negative: 227565
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.437567 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16299
[LightGBM] [Info] Number of data points in the train set: 454904, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499752 -> initscore=-0.000994
[LightGBM] [Info] Start training from score -0.000994
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-04-12 10:02:25,257] Trial 9 finished with value: 0.9987319574638043 and parameters: {'n_estimators': 133, 'max_depth': 4, 'learning_rate': 0.17352347161986317}. Best is trial 6 with value: 0.9992150257352622.
[I 2025-04-12 10:02:25,259] A new study created in memory with name: no-name-29358f81-5633-4897-a056-15ee534ab002
[I 2025-04-12 10:03:17,971] Trial 0 finished with value: 0.9997106211381087 and parameters: {'iterations': 172, 'depth': 8, 'learning_rate': 0.273391014011567}. Best is trial 0 with value: 0.9997106211381087.
[I 2025-04-12 10:03:59,499] Trial 1 finished with value: 0.9994668238124979 and parameters: {'iterations': 130, 'depth': 8, 'learning_rate': 0.13304279166008937}. Best is trial 0 with value: 0.9997106211381087.
[I 2025-04-12 10:04:26,844] Trial 2 finished with valu

[LightGBM] [Info] Number of positive: 227339, number of negative: 227565
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.534127 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16299
[LightGBM] [Info] Number of data points in the train set: 454904, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499752 -> initscore=-0.000994
[LightGBM] [Info] Start training from score -0.000994


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


AUC: 0.9997
AUPRC: 0.9997


In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Threshold predictions at 0.5 to get binary class predictions
y_pred = (y_pred_proba >= 0.5).astype(int)

# Metrics
auc = roc_auc_score(y_test, y_pred_proba)
auprc = average_precision_score(y_test, y_pred_proba)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Display results
print(f"AUC:    {auc:.4f}")
print(f"AUPRC:  {auprc:.4f}")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")


AUC:    0.9997
AUPRC:  0.9997
Accuracy:  0.9938
Precision: 0.9946
Recall:    0.9931
F1 Score:  0.9938
